# FCN

## 1. Environment Setup

### 1.0. Check GPU

In [1]:
!nvidia-smi

Sun Jan 31 06:06:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:01:00.0 Off |                  N/A |
| 76%   60C    P2   138W / 370W |    281MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:03:00.0 Off |                  N/A |
| 25%   

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


### 1.1. Install Pytorch and torchvision

In [3]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.2. Install scipy

In [4]:
!pip install scipy

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### 1.3. Import

In [5]:
import augmentation, train

import torch
import torchvision

import matplotlib.pyplot as plt

# Ignore Warning
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Preprocessing

In [6]:
train_batch_size = 1
test_batch_size = 1

In [7]:
train_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                             augmentation.Resize((256, 256)), augmentation.RandomCrop((224, 224)), 
                                             augmentation.RandomHorizontalFlip(),
                                             augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                           augmentation.Resize((256, 256)), 
                                           augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = torchvision.datasets.SBDataset(root='./', image_set='train_noval', mode='segmentation', download=False, transforms=train_tf)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)

val_dataset = torchvision.datasets.VOCSegmentation(root='./', year='2012', image_set='val', download=True, transforms=val_tf)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

Using downloaded and verified file: ./VOCtrainval_11-May-2012.tar


## 3. Train Model

In [8]:
num = 32
num_classes = 21
ignore_index = 255

gpu_id = 0
print_freq = 1500
epoch_print = 2

save = False
epochs = 36

lr = 0.0001
momentum = 0.99
weight_decay = 0.0005

In [9]:
fcn = train.FCN(num=num, num_classes=num_classes, ignore_index=ignore_index, 
                gpu_id=gpu_id, print_freq=print_freq, epoch_print=epoch_print)

In [10]:
fcn.train(train_loader, val_loader, save=save, epochs=epochs, lr=lr, momentum=momentum, weight_decay=weight_decay)

Epoch 1 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 1500 - Train Loss : 0.719932, Test Loss : 0.884782, Test mIoU : 10.3253


*********************************** Best mIoU Updated ***********************************
Iteration : 3000 - Train Loss : 0.453612, Test Loss : 0.883543, Test mIoU : 14.7557


*********************************** Best mIoU Updated ***********************************
Iteration : 4500 - Train Loss : 0.562616, Test Loss : 0.662406, Test mIoU : 22.1239

Epoch 3 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 1500 - Train Loss : 0.713833, Test Loss : 0.492267, Test mIoU : 40.5549

Iteration : 3000 - Train Loss : 0.303383, Test Loss : 0.620168, Test mIoU : 33.8692
Iteration : 4500 - Train Loss : 0.670539, Test Loss : 0.488844, Test mIoU : 40.5482
Epoch 5 Started...

*********************************** Best mIoU Updated ****************